In [ ]:
from ImageLab import *
import numpy as np
from PIL import Image
import cv2

In [ ]:
CustomGx_img2 = Image.open(
    'Image/Powerline/Custom_Gx_Simple.png').convert('RGB')
CustomGx_img2 = np.array(CustomGx_img2)

CustomGy_img2 = Image.open(
    'Image/Powerline/Custom_Gy_Simple.png').convert('RGB')
CustomGy_img2 = np.array(CustomGy_img2)

Robert_Gx_img2 = Image.open(
    'Image/Powerline/Robert_Gx_Simple.png').convert('RGB')
Robert_Gx_img2 = np.array(Robert_Gx_img2)

Robert_Gy_img2 = Image.open(
    'Image/Powerline/Robert_Gy_Simple.png').convert('RGB')
Robert_Gy_img2 = np.array(Robert_Gy_img2)

CustomGradient = EdgeDetect().gradient_mag(CustomGx_img2, CustomGy_img2)
#ImagePlotter(CustomGradient).plot_image('Custom_Gradient')

RobertGradient = EdgeDetect().gradient_mag(Robert_Gx_img2, Robert_Gy_img2)
#ImagePlotter(RobertGradient).plot_image('Roberts_Gradient')

# Eq_CustomGradient = Filters(CustomGradient).histogram_equalization()
# ImagePlotter(Eq_CustomGradient).plot_image('Eq_Custom_Gradient')

# Eq_CustomGradient = Filters(CustomGradient).gamma_correction(3)
# ImagePlotter(Eq_CustomGradient).plot_image('Gamma_Custom_Gradient')


In [ ]:

CustomGradient_Simple, _ = Segment(
    CustomGradient).global_multiple_threshold((0, 80))


def expand_dims(img):
    if np.array(img).ndim == 2:
        return np.expand_dims(img, axis=2)


gray = ColorSpace(CustomGradient_Simple).rgb_to_grayscale()
gray = expand_dims(gray)
print(gray.shape)

gray = np.uint8(gray)
Simple_Canny = cv2.Canny(gray, 50, 150, apertureSize=3)
ImagePlotter(Simple_Canny).plot_image('Simple_Canny')


In [ ]:
lines = cv2.HoughLines(Simple_Canny, 1, np.pi * 2 / 180, 160)
print(lines.shape)


def DrawLines(img, lines):
    # Draw the detected lines on the original image
    img = np.copy(img)
    for line in lines:
        rho, theta = line[0]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * a)
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * a)
        cv2.line(img, (x1, y1), (x2, y2), (255, 255, 255), thickness=2)

    return img


img = DrawLines(Simple_Canny, lines)
ImagePlotter(img).plot_image('Simple_Canny_with_Lines')


In [ ]:
def get_lines_coordinates(img, hough_lines):
    lines = []

    base_x, base_y = 0, img.shape[0] - 1

    for line in hough_lines:
        rho, theta = line[0]
        a = np.cos(theta)
        b = np.sin(theta)

        # Calculate new x and y values relative to the base point
        x1 = int(a * -rho) + base_x
        y1 = int(b * rho) + base_y
        x2 = int(a * (-rho + img.shape[1])) + base_x
        y2 = int(b * (rho - img.shape[1])) + base_y

        lines.append([(x1, y1), (x2, y2)])

    return lines


def DrawCoords(img, hough_lines):

    img = np.copy(img)
    lines = get_lines_coordinates(img, hough_lines)

    # Loop through coordinate list and plot onto the image
    for line in lines:
        cv2.circle(img, tuple(line[0]), 10, (0, 0, 255), -1)
        cv2.circle(img, tuple(line[1]), 10, (0, 0, 255), -1)
        print(line)

    return img


In [ ]:
img = DrawCoords(Simple_Canny, lines)
ImagePlotter(img).plot_image('Simple_Canny_with_Lines')

In [ ]:
Tilation(img).split_image_nxn_sections(30)